<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/continuous_outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Fragility Metrics Toolkit: Continuous Outcomes
# 28-NOV-2025
# Fully aligned with FRAGILITY_METRICS.md v10.3.1 §3.7 CFQ/CFS and §4.3 MeCI
#
# Three modes:
#   MODE 1: Full data independent groups (m1, sd1, n1, m2, sd2, n2) — Welch t-test
#   MODE 2: CI-only (mean difference + 95% CI bounds) — z-approximation
#   MODE 3: Paired/matched data (d_mean, sd_diff, n_pairs) — Paired t-test
#
# Output: p (t-test p-value), fr (CFS/CFQ), nb (MeCI)
#
# Formulas (exact v10.3.1):
# - CFS  = ||T| – t*|  → always positive distance to α=0.05 boundary
# - CFQ  = CFS / (1 + CFS)
# - MeCI = |T| / (1 + |T|) where T is the t-statistic
#
# Note: MODE 1 & 3 require standard deviations (SD), not standard errors (SE).
# Most papers report mean ± SE. Convert using: SD = SE × √n
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

import numpy as np
from scipy.stats import t

ALPHA = 0.05

# ==================== MODE 1: Full data (m1, sd1, n1, m2, sd2, n2) ====================
def full_data_mode(m1, sd1, n1, m2, sd2, n2):
    """
    Calculate p-fr-nb from complete group statistics (independent samples).
    Requires: means, standard deviations, and sample sizes for both groups.
    Uses Welch t-test (allows unequal variances).
    """
    if n1 < 2 or n2 < 2 or sd1 <= 0 or sd2 <= 0:
        raise ValueError("Invalid input: n ≥ 2 and SD > 0 required")

    # Welch t-test
    se1 = sd1 / np.sqrt(n1)
    se2 = sd2 / np.sqrt(n2)
    se_diff = np.sqrt(se1**2 + se2**2)
    t_stat = (m1 - m2) / se_diff
    df = (se1**2 + se2**2)**2 / (se1**4 / (n1-1) + se2**4 / (n2-1))
    p_val = 2 * t.sf(np.abs(t_stat), df)
    t_crit = t.ppf(1 - ALPHA/2, df)

    # CFQ / CFS — symmetric distance to boundary
    cfs = abs(abs(t_stat) - t_crit)
    cfq = cfs / (1 + cfs)

    # MeCI — exact v10.3.1 Neutrality Boundary Framework
    meci = abs(t_stat) / (1 + abs(t_stat))

    # Calculate 95% CI for the mean difference
    mean_diff = m1 - m2
    ci_lower = mean_diff - t_crit * se_diff
    ci_upper = mean_diff + t_crit * se_diff

    return {
        "p": p_val,
        "CFS": cfs,
        "CFQ": cfq,
        "MeCI": meci,
        "T": t_stat,
        "df": df,
        "se_diff": se_diff,
        "mean_diff": abs(mean_diff),
        "CI_lower": ci_lower,
        "CI_upper": ci_upper
    }

# ==================== MODE 2: CI-only (mean difference + 95% CI) ====================
def ci_only_mode(delta, ci_lower, ci_upper):
    """
    Calculate approximate p-fr-nb from mean difference and 95% CI only.
    Uses large-sample z-approximation (t* ≈ 1.96).
    """
    # Extract SE from CI width
    se_diff = (ci_upper - ci_lower) / 3.92  # 95% CI width ÷ (2 × 1.96)

    # Approximate T-statistic
    T_approx = abs(delta) / se_diff
    t_crit = 1.96  # large-sample approximation

    # CFS (corrected with nested absolute value)
    cfs = abs(abs(T_approx) - t_crit)
    cfq = cfs / (1 + cfs)

    # MeCI
    meci = T_approx / (1 + T_approx)

    # Approximate p-value
    p_approx = 2 * (1 - t.cdf(T_approx, df=999))

    return {
        "p": p_approx,
        "CFS": cfs,
        "CFQ": cfq,
        "MeCI": meci,
        "T": T_approx,
        "se_diff": se_diff,
        "mean_diff": abs(delta),
        "CI_lower": ci_lower,
        "CI_upper": ci_upper
    }

# ==================== MODE 3: Paired data (d_mean, sd_diff, n) ====================
def paired_data_mode(d_mean, sd_diff, n):
    """
    Calculate p-fr-nb from paired/matched data.
    Requires: mean of paired differences, SD of differences, number of pairs.

    Typical use cases:
    - Crossover trials (same subjects receive both treatments)
    - Pre-post measurements (same subjects measured twice)
    - Matched pairs (twins, matched controls)
    """
    if n < 2 or sd_diff <= 0:
        raise ValueError("Invalid input: n ≥ 2 and SD > 0 required")

    # Paired t-test
    se_diff = sd_diff / np.sqrt(n)
    t_stat = d_mean / se_diff
    df = n - 1  # Paired data uses n-1, not Welch-Satterthwaite
    p_val = 2 * t.sf(np.abs(t_stat), df)
    t_crit = t.ppf(1 - ALPHA/2, df)

    # CFQ / CFS — symmetric distance to boundary
    cfs = abs(abs(t_stat) - t_crit)
    cfq = cfs / (1 + cfs)

    # MeCI — exact v10.3.1 Neutrality Boundary Framework
    meci = abs(t_stat) / (1 + abs(t_stat))

    # Calculate 95% CI for the mean difference
    ci_lower = d_mean - t_crit * se_diff
    ci_upper = d_mean + t_crit * se_diff

    return {
        "p": p_val,
        "CFS": cfs,
        "CFQ": cfq,
        "MeCI": meci,
        "T": t_stat,
        "df": df,
        "se_diff": se_diff,
        "mean_diff": abs(d_mean),
        "CI_lower": ci_lower,
        "CI_upper": ci_upper
    }

# ==================== Output formatter ====================
def print_results(results, mode="full"):
    # Fragility interpretation
    if results['CFQ'] < 0.05:
        fr_interp = "FRAGILE"
    else:
        fr_interp = "STABLE"

    # Robustness interpretation
    if results['MeCI'] < 0.075:
        nb_interp = "CLOSE TO NEUTRALITY / WEAK"
    elif results['MeCI'] < 0.227:
        nb_interp = "MODERATE SEPARATION"
    else:
        nb_interp = "FAR FROM NEUTRALITY / STRONG"

    print("\n" + "="*60)
    if mode == "full":
        print("FULL DATA MODE: INDEPENDENT GROUPS (Welch t-test)")
    elif mode == "paired":
        print("PAIRED DATA MODE: MATCHED/CROSSOVER DESIGN")
    else:
        print("CONTINUOUS OUTCOMES: DIFFERENCE + 95% CONFIDENCE INTERVALS")
    print("="*60)

    print(f"Mean difference = {results['mean_diff']:.4f} (95% CI: {results['CI_lower']:.4f} to {results['CI_upper']:.4f})")
    print("-"*60)
    print("\nCOMPLETE p-fr-nb TRIPLET (significance, fragility, robustness):")
    print("-" * 60)

    if mode in ["full", "paired"]:
        print(f"df = {results['df']:.1f}")

    print(f"p-value = {results['p']:.6f}  →  {'significant' if results['p']<=ALPHA else 'not significant'}")
    print(f"fr: CFQ = {results['CFQ']:.6f}  (fragility: {fr_interp})")
    print(f"    CFS = {results['CFS']:.6f} SE-units")
    print(f"nb: MeCI = {results['MeCI']:.6f}  (robustness: {nb_interp})")

    print("\nCitation:")
    print("Heston, T. F. (2025). Fragility Metrics Toolkit [Software].")
    print("Zenodo. https://doi.org/10.5281/zenodo.17254763")
    print()
    print("Report Bugs: https://faculty.washington.edu/theston/contact")
    print("="*70 + "\n")

# ==================== Example usage ====================
if __name__ == "__main__":
    print("Calculator: continuous_outcomes\n")

    print("Select mode:")
    print("  1 = Independent groups (full data: m1, sd1, n1, m2, sd2, n2)")
    print("  2 = CI only (mean difference + 95% CI)")
    print("  3 = Paired/matched data (mean difference, SD of differences, n pairs)\n")

    mode = input("Choose mode (1/2/3): ")

    if mode == "1":
        print("\nFull data mode - enter group statistics:")
        m1 = float(input("Group 1 mean: "))
        sd1 = float(input("Group 1 SD: "))
        n1 = int(input("Group 1 n: "))
        m2 = float(input("Group 2 mean: "))
        sd2 = float(input("Group 2 SD: "))
        n2 = int(input("Group 2 n: "))

        results = full_data_mode(m1, sd1, n1, m2, sd2, n2)
        print_results(results, mode="full")

    elif mode == "2":
        print("\nCI-only mode - enter mean difference and 95% CI:")
        delta = float(input("Mean difference: "))
        ci_lower = float(input("95% CI lower bound: "))
        ci_upper = float(input("95% CI upper bound: "))

        results = ci_only_mode(delta, ci_lower, ci_upper)
        print_results(results, mode="ci")

    elif mode == "3":
        print("\nPaired data mode - enter paired statistics:")
        d_mean = float(input("Mean of paired differences: "))
        sd_diff = float(input("SD of paired differences: "))
        n = int(input("Number of pairs: "))

        results = paired_data_mode(d_mean, sd_diff, n)
        print_results(results, mode="paired")

    else:
        print("Invalid mode selection. Use 1, 2, or 3.")

Calculator: continuous_outcomes

Select mode:
  1 = Independent groups (full data: m1, sd1, n1, m2, sd2, n2)
  2 = CI only (mean difference + 95% CI)
  3 = Paired/matched data (mean difference, SD of differences, n pairs)

